In [ ]:
#E1: With the data having more variables than observations using OLS could lead to overfitting as there may be more than one solution and be subject to higher variance. 
#    To solve this I would use a ridge regression or LASSO model that way the bias would slightly increase to largly decrease variace or cut out uncessesary variables.  
#E2: F
#E3: B
#E4: High Bias, reduce the regularization parameter

In [12]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import statistics

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'CarPrice_Assignment.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
car_price = pd.read_csv(file_content_stream)
car_price.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
# defining input and target variables 
x = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm','citympg', 'highwaympg']]
y = car_price['price']

# list to store coefficents 
coef = list()

for i in range(0, 1000): 
    # splitting the data into train and test 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)
    
    # running LASSO cross-validation to estimate optimal lambda
    lasso_cv = LassoCV(normalize = True, cv = 5).fit(x_train, y_train)
    
    # building LASSO regression with optimal lambda
    lasso_md = Lasso(alpha = lasso_cv.alpha_, normalize = True).fit(x_train, y_train)
    
    # storing estimated coefficients
    coef.append(lasso_md.coef_)
    
# putting coefficient list in data-frame
coef_data = pd.DataFrame(coef)

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing

In [5]:
coef_data.head()

,0,1,2,3,4,5,6
0,219.313167,102.954821,213.268982,49.236509,1.540003,-60.212967,-0.000000
1,181.633782,111.298544,280.033213,37.372812,2.182260,-133.471765,-0.000000
2,129.845390,116.173991,327.125394,39.637905,1.867805,-170.830107,-0.000000
3,167.605991,99.808382,334.772780,43.468509,1.515528,-147.850684,-28.867938
4,128.870423,109.751418,292.774225,49.696192,2.099746,-116.848567,-0.000000


In [6]:
sum(coef_data[6] == 0.0)

721

In [7]:
# defining input and target variables 
x = car_price[['wheelbase', 'enginesize', 'compressionratio', 'horsepower', 'peakrpm','citympg']]
y = car_price['price']

# splitting the data into train and test 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)

# Normalizing the data
def l2_normalization(x):
    x_mean = np.mean(x)
    l2 = np.sqrt(sum(x**2))
    return (x - x_mean) / l2

x_train = x_train.apply(l2_normalization, axis = 1)
x_test = x_test.apply(l2_normalization, axis = 1)

In [8]:
# building the linear model 
lm_md = LinearRegression().fit(x_train, y_train)

# predicting on test data
lm_pred = lm_md.predict(x_test)

# computing the mse
print('The mse of the linear regression model is:', np.mean((y_test - lm_pred)**2))

The mse of the linear regression model is: 20798274.223668143


In [14]:
# finding best lambda over 100 times
lambda_list = list()

for i in range(0,100):
    # ridge regression
    ridge_cv = RidgeCV(alphas =  [.001, .01, .1, 1, 10, 100], cv = 5).fit(x_train, y_train)

    # Extracting
    cv_lambda = ridge_cv.alpha_
    
    #storing lambda
    lambda_list.append(cv_lambda)
    
# finding most frequent lambda
best_lambda = statistics.mode(lambda_list)

print('the best lambda of the ridge model is:', best_lambda)

the best lambda of the ridge model is: 0.001


In [15]:
# building the model
ridge_md = Ridge(alpha = best_lambda).fit(x_train, y_train)

# predicting on the test data
ridge_pred = ridge_md.predict(x_test)

# computing the mse
print('the mse of the ridge model is:', np.mean(np.power(y_test - ridge_pred, 2)))

the mse of the ridge model is: 23241997.889556546


In [ ]:
# the mse of the linear model is smaller, so it would be prefered to predict car price